In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import ipywidgets as wg
from IPython.core.display import HTML
import matplotlib.animation as anim
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
scouting_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
scouting_df.head()
#list(scouting_df.columns)

In [ ]:
# Keep the columns that are related to kickoffs
kickoff_cols = ['gameId', 'playId', 'hangTime', 'kickType', 'kickDirectionIntended', 'kickDirectionActual', 'missedTackler',
                'returnDirectionIntended', 'returnDirectionActual', 'kickoffReturnFormation', 'specialTeamsSafeties']
kickoff_scouting_df = scouting_df[kickoff_cols]
kickoff_scouting_df

In [ ]:
plays_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/plays.csv')
kickoffs_df = plays_df.loc[plays_df['specialTeamsPlayType'] == 'Kickoff']
kickoff_returns_df = kickoffs_df.loc[kickoffs_df['specialTeamsResult'] == 'Return']
kickoff_returns_df

In [ ]:
# Merge the scouting data with the kickoffs that were returned.
kickoff_returns_df = kickoff_returns_df.merge(kickoff_scouting_df, how='inner', on=['gameId', 'playId'])
kickoff_returns_df
#list(kickoff_returns_df.columns)

In [ ]:
tracking2018 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2018.csv')
tracking2019  = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2019.csv')
tracking2020 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2020.csv')

In [ ]:
tracking_data = tracking2018.append(tracking2019).append(tracking2020)
tracking_cols = ['s', 'a', 'nflId', 'gameId','playId', 'frameId', 'event', "x", "y", "time", "team", "dis"]
tracking_data = tracking_data[tracking_cols]
tracking_data 

In [ ]:
tracking = tracking[tracking['nflId'].isin(list(punt_return['returnerId'].unique()))]

In [ ]:
players_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/players.csv') 
positions = ['WR','RB','DB']
sub_players_df = pd.DataFrame(players_df[players_df.Position.isin(positions)])

sub_players_df['nflId'] = sub_players_df['nflId'].astype(int)
sub_players_df

In [ ]:
dropped_returns = kickoff_returns_df.loc[kickoff_returns_df['returnerId'].str.contains(';', na = False)]
kickoff_returns_df = kickoff_returns_df[~kickoff_returns_df.isin(dropped_returns)]
kickoff_returns_df['returnerId'] = pd.to_numeric(kickoff_returns_df['returnerId'])
kickoff_returns_df = kickoff_returns_df.dropna(subset=['returnerId'])
kickoff_returns_df['returnerId'] = kickoff_returns_df['returnerId'].astype(int)
kickoff_returns_players_df = pd.merge(kickoff_returns_df,sub_players_df,
                             how='left',left_on ='returnerId',right_on ='nflId')
kickoff_returns_players_df = kickoff_returns_players_df.loc[:,~kickoff_returns_players_df.columns.duplicated()]
kickoff_returns_players_df


1. Analyze whether teams that featured receivers most often scored more touchdowns or gained more yards. (ID average number of receivers)
2. Examine kickoffs that went for greater than 50 yds (2018-2020) and see what position group achieved the most
3. Analyze which position group performed the best in TDs, YPR, Total Yardage
4. Analyze which players per position group performed the best in TDs, YPR, Total Yardage


In [ ]:
no_na_kickoff_returns_players_df = kickoff_returns_players_df.loc[kickoff_returns_players_df['possessionTeam'] > 'NaN']
no_na_kickoff_returns_players_df

no_na_kickoff_returns_players_df['Position'].value_counts().plot.barh(figsize=(9,3), title="Postition Count")


In [ ]:
td_returns = no_na_kickoff_returns_players_df.loc[no_na_kickoff_returns_players_df['playDescription'].str.contains('TOUCHDOWN')]
td_returns
td_returns['Position'].value_counts().plot.barh(figsize=(9,3), title="Postition Count by touchdowns")


In [ ]:
fifty_kickoff_returns_players_df = no_na_kickoff_returns_players_df.loc[kickoff_returns_players_df['kickReturnYardage'] >= 50.0]
fifty_kickoff_returns_players_df.head()

fifty_kickoff_returns_players_df['Position'].value_counts().plot.barh(figsize=(9,3), title="Postition Count by greater than 50 yards")

In [ ]:
fifty_kickoff_returns_players_df['Year'] = fifty_kickoff_returns_players_df['gameId'].astype(str).str[0:4]
fifty_kickoff_returns_players_df.head()

ax = fifty_kickoff_returns_players_df.groupby(['Position','Year']).size().plot.barh(figsize=(14,6), title="Kickoff returns greater than 50 yards by Position Group")
ax.set_ylabel('Kicking Position')

test = fifty_kickoff_returns_players_df.groupby(['Year','Position'])
test.first()

#fifty_kickoff_returns_players_df['Position'].value_counts().plot.barh(figsize=(9,3), title="Postition Count by greater than 50 yards")

In [ ]:
no_na_kickoff_returns_players_df.groupby(['Position','displayName'])['kickReturnYardage'].mean()
no_na_kickoff_returns_players_df.groupby(['Position','displayName']).nunique()
#td_returns['Position'].count()



In [ ]:
no_na_kickoff_returns_players_df.groupby(['Position','displayName'])['kickReturnYardage'].mean()